<h1>Churn-Detection Using ANN</h1>

<h2>Load Libraries</h2>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

<h2>Importing Telco Customer Churn Dataset</h2>

https://www.kaggle.com/blastchar/telco-customer-churn

In [2]:
df=pd.read_csv('Churn-dataset.csv')
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4849,8190-ZTQFB,Male,0,No,No,41,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,One year,Yes,Electronic check,51.35,2075.1,No
1323,4565-NLZBV,Female,0,Yes,No,71,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),24.65,1710.15,No
1472,9283-LZQOH,Male,0,Yes,Yes,13,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Credit card (automatic),74.40,896.75,Yes
750,8224-IVVPA,Female,0,No,No,27,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,56.70,1652.95,No
3269,2662-NNTDK,Male,0,No,No,65,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Credit card (automatic),24.80,1600.95,No


<h2>Data Preprocessing</h2>

In dataset we can see there are many unwanted things.We'll remove them and clean our data

In [3]:
df.drop('customerID',axis='columns',inplace=True)
df = df.dropna(how='all') # remove samples with all missing values
df = df[~df.duplicated()] # remove duplicates
df.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
102,Male,0,No,No,38,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,No,One year,No,Bank transfer (automatic),95.0,3605.6,No
5891,Male,0,No,No,11,No,No phone service,DSL,No,No,Yes,No,Yes,No,One year,Yes,Electronic check,41.6,470.6,Yes
1957,Female,0,Yes,No,46,Yes,Yes,DSL,Yes,No,No,No,No,No,Month-to-month,No,Electronic check,53.1,2459.8,No
3404,Female,0,No,Yes,35,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Credit card (automatic),55.6,2016.45,No
2350,Female,0,No,No,5,Yes,Yes,Fiber optic,No,No,No,No,Yes,No,Month-to-month,Yes,Electronic check,84.0,424.75,No


In [4]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

We can see that 'TotalCharges' is of type object,So lets change it to float

In [5]:
#changing TotalCharges from object to float
total_charges_filter = df.TotalCharges == " "
df = df[~total_charges_filter]
df.TotalCharges = pd.to_numeric(df.TotalCharges)
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

<h3>Splitting Dataset into inputs(X) and targets(Y) </h3>

In [6]:
X=df.iloc[:,:19]
Y=df.iloc[:,19:]
X.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [7]:
Y.head()

,Churn
0,No
1,No
2,Yes
3,No
4,Yes


<h3>Converting all values to integer or float</h3>

In [8]:
Y['Churn'].replace({'Yes': 1,'No': 0},inplace=True)
X.replace('No internet service','No',inplace=True)
X.replace('No phone service','No',inplace=True)
X['gender'].replace({'Female':1,'Male':0},inplace=True)

In [9]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling',]
for col in yes_no_columns:
    X[col].replace({'Yes': 1,'No': 0},inplace=True)

<h3>Converting Categorical Data</h3>

In [10]:
X = pd.get_dummies(data=X, columns=['InternetService','Contract','PaymentMethod'])
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,1,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,34,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,2,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,45,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,2,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


<strong>Removing one column from each categorical column to avoid Dummy Variable Trap</strong>

In [11]:
X.drop('InternetService_No',axis='columns',inplace=True)
X.drop('Contract_Month-to-month',axis='columns',inplace=True)
X.drop('PaymentMethod_Bank transfer (automatic)',axis='columns',inplace=True)

In [12]:
X.dtypes

gender                                     int64
SeniorCitizen                              int64
Partner                                    int64
Dependents                                 int64
tenure                                     int64
PhoneService                               int64
MultipleLines                              int64
OnlineSecurity                             int64
OnlineBackup                               int64
DeviceProtection                           int64
TechSupport                                int64
StreamingTV                                int64
StreamingMovies                            int64
PaperlessBilling                           int64
MonthlyCharges                           float64
TotalCharges                             float64
InternetService_DSL                        uint8
InternetService_Fiber optic                uint8
Contract_One year                          uint8
Contract_Two year                          uint8
PaymentMethod_Credit

<h3>Splitting dataset into Test and Train </h3>

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [14]:
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,MonthlyCharges,TotalCharges,InternetService_DSL,InternetService_Fiber optic,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
1251,1,0,0,0,1,1,0,0,1,0,...,0,75.80,75.80,0,1,0,0,0,1,0
676,1,0,0,0,48,1,1,0,0,0,...,0,24.55,1203.95,0,0,1,0,0,0,0
381,0,0,0,0,1,1,0,0,1,0,...,0,50.65,50.65,1,0,0,0,0,0,1
2313,0,0,1,1,43,1,1,0,0,1,...,1,104.30,4451.85,0,1,1,0,1,0,0
6901,1,0,0,0,30,1,1,0,1,1,...,1,68.95,2038.70,1,0,1,0,1,0,0


In [15]:
Y_train.head()

,Churn
1251,1
676,0
381,1
2313,0
6901,0


<h2>Training ANN Model for Churn-Detection</h2>

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [17]:
model = Sequential()
model.add(Dense(23,  kernel_initializer='uniform', input_shape=(23,), activation='relu'))
model.add(Dense(23,  kernel_initializer='uniform', input_shape=(23,), activation='relu'))
model.add(Dense(1,  kernel_initializer='uniform', input_shape=(23,), activation='sigmoid'))


In [18]:
model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=10 ,epochs=100)


Epoch 1/100
561/561 [==============================] - 2s 3ms/step - loss: 0.5757 - accuracy: 0.7475
Epoch 2/100
561/561 [==============================] - 1s 3ms/step - loss: 0.5167 - accuracy: 0.7805
Epoch 3/100
561/561 [==============================] - 2s 3ms/step - loss: 0.4618 - accuracy: 0.7863
Epoch 4/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4422 - accuracy: 0.8014
Epoch 5/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4476 - accuracy: 0.7955
Epoch 6/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4352 - accuracy: 0.7926
Epoch 7/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4174 - accuracy: 0.8122
Epoch 8/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4393 - accuracy: 0.7951
Epoch 9/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4438 - accuracy: 0.7865
Epoch 10/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4293 - accura

561/561 [==============================] - 1s 3ms/step - loss: 0.4136 - accuracy: 0.7966
Epoch 82/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4247 - accuracy: 0.7977
Epoch 83/100
561/561 [==============================] - 1s 3ms/step - loss: 0.3928 - accuracy: 0.8136
Epoch 84/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4115 - accuracy: 0.8056
Epoch 85/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4065 - accuracy: 0.8070: 0s - loss: 0.4065 - accuracy: 0.80
Epoch 86/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4168 - accuracy: 0.8067
Epoch 87/100
561/561 [==============================] - 1s 3ms/step - loss: 0.3999 - accuracy: 0.8108
Epoch 88/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4042 - accuracy: 0.8108
Epoch 89/100
561/561 [==============================] - 1s 3ms/step - loss: 0.4047 - accuracy: 0.8064
Epoch 90/100
561/561 [==============================] - 1s 

<strong>Saving model in model.h5 file</strong>

In [ ]:
model.save("model.h5")

In [ ]:
#model with acuuracy of 80.8% if needed
# model=keras.models.load_model("final_model.h5")
# print(m)

<h3>Evaluating our ANN model</h3>

In [ ]:
Y_pred=model.predict(X_test)
Y_pred=(Y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(Y_test,Y_pred)
print(classification_report(Y_test,Y_pred))

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
#accuracy=(TP+TN)/total
(901+216)/1407